# 北尾早霧・砂川武貴・山田知明『定量的マクロ経済学と数値計算』日本評論社
## 第１章：数値計算ことはじめ
* こちらのNotebookではJuliaの基本的な知識と関連付けて、数値計算誤差について説明をしています。
    * 数値計算誤差についてはテキストの1.7節を参照してください。

---

* 本Notebookを作成した際の環境は以下の通りです。
* 計算速度などについては使用するPCによって違いがあります。
* また、Juliaのバージョンによって、多少コマンドに違いがある可能性があります。
    * 特別な関数は使っていないはずですが。

In [1]:
versioninfo()

Julia Version 1.7.3
Commit 742b9abb4d (2022-05-06 12:58 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin21.4.0)
  CPU: Intel(R) Core(TM) i7-8569U CPU @ 2.80GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, skylake)


---

## Juliaにおける型(Type)
- Juliaは**動的型付け (dynamic typing)** の言語なので変数の**型（type）**を気にする必要は基本的にはない
- 型とはなにか?
    - 例えば、コンピュータ上は3と3.0は別物として区別する
    - 前者は**整数 (Integer)** で後者は**浮動小数点 (Floating Point Number: Float)** と呼ばれる
    - MatlabやJulia、Pythonなど(動的型付けの言語)ではあまり深く考えなくても、多くの場合、直感的に処理をしてくれる
        - 例1：3 + 3.0 = 6.0
        - 例2：2 + 2.1 = 4.1
        - ただし**常に**自分の思っているとおりに処理してくれるとは限らないかも
    - C言語やFortranなどの**静的型付け (static typing)** の言語では必ず事前に変数の型を決めておく必要がある(**宣言**)
        - integer a = 3 ←整数として変数aを定義
        - float b = 3.0 ←浮動小数点として変数bを定義
        - ⬆はイメージで実際のFortranやC言語の記法ではない
    - a + bを計算しようとすると「型が違う！」と怒られる
    - 浮動小数点とは何かをしっかり学びたい場合、例えばクアルテローニ他(2014)の第1章を参照
- 加えて、コンピュータでは数字の表現の仕方(**2進法**)によって、表示できる桁数に違いがある
    - 8ビット、16ビット、32ビット、64ビットで最大桁数が異なる
        - 初代のファミコンは8ビット、スーパーファミコンは16ビット、プレイステーションは32ビットとだんだん性能が上がっていった！
    - それぞれInt32、Int64、Float64みたいな感じで表現される
- Juliaではデフォルトで64ビットになる：⬇を参照
    - かなり古いPCを使っている場合この限りではないが、めったにいないはず
- Juliaでは整数+浮動小数点は浮動小数点に揃えてくれる
    - 掛け算や割り算なども自動で調整してくれるけど、ちゃんとルールを把握しないで異なる型同士の演算は**避けたほうが無難**

In [1]:
# 整数
a = 3

3

In [2]:
# 型を確認する
typeof(a)

Int64

In [3]:
# 浮動小数点
b = 3.0

3.0

In [4]:
typeof(b)

Float64

In [5]:
a + b

6.0

In [6]:
typeof(a + b)

Float64

---

## 例：整数と浮動小数点の違いを気にしなかった場合に問題が起こる事例
- べき乗の計算では整数を使うか、浮動小数点を使うかで計算速度が変わる
- 例：5の2乗と5の2.0乗は計算速度が異なる
    - 当然、数学上は全く同じ
    - コンピュータ上の計算処理の仕方が異なるため
    - 3倍程度、計算速度が遅くなっている

In [7]:
c = 2

2

In [8]:
d = 2.0

2.0

* ループ(繰り返し計算)についてはまた後ほど説明予定
* @timeは計算速度を計測してくれる**マクロ**

In [9]:
# 5の2乗を1000万回計算してみる
@time for i = 1:10000000
    ans = 5^c
end

  0.139026 seconds (15 allocations: 864 bytes, 1.66% compilation time)


In [10]:
# 5の2.0乗を1000万回計算してみる：こちらのほうが遅い
@time for i = 1:10000000
    ans = 5^d
end

  0.399429 seconds (10.01 M allocations: 152.855 MiB, 4.58% gc time, 1.79% compilation time)


In [11]:
# 直書きが一番はやい：なぜ?
@time for i = 1:10000000
    ans = 5^2
end

  0.000001 seconds


---

## Tips：書き方で計算速度が変わったりする
- 変数(tmp)を事前に用意してそこに値を代入していく場合、**代入という行為そのもの**に計算時間がかかる
    - tmpは1000万個の要素を持つ**配列(ベクトル)**
- ⬆の例だと5の2乗を1000万回計算して、その結果はすべて捨てていた
    - 途中の計算結果を取り出すことは出来ない
- ⬇の２つの例の場合、変数の代入にかかる時間の方が影響が大きくて、「指数が整数か浮動小数点か」の影響は相対的に小さい事がわかる
    - それでもわかっているのであれば、指数は整数で定義すべき
- **注意**：計算速度自体は使っているPCのスペックに依存する

In [12]:
# 5の2乗を1000万回計算してみる
num_len = 10000000
tmp = zeros(Int64, num_len) # 要素の数がnum_len個のベクトル(配列)を準備：中身はすべて0
@time for i = 1:num_len
    tmp[i] = 5^c
end

  0.963350 seconds (30.00 M allocations: 610.336 MiB, 15.84% gc time)


In [13]:
# 5の2.0乗を1000万回計算してみる
num_len = 10000000
tmp = zeros(num_len)
@time for i = 1:num_len
    tmp[i]= 5^d
end

  1.102978 seconds (40.00 M allocations: 762.925 MiB, 5.23% gc time, 0.23% compilation time)


In [14]:
# やはり直書きが一番早い
num_len = 10000000
tmp = zeros(Int64, num_len) # 要素の数がnum_len個の配列
@time for i = 1:num_len
    tmp[i] = 5^2
end

  0.854587 seconds (30.00 M allocations: 610.336 MiB, 21.61% gc time)


---

## 型ごとに使える最大値・最小値
- ビット数が大きい方が扱える数字の範囲が大きくなる
    - 昔のゲームでHPの上限が255だったり、65535だったりした理由
- 今のコンピュータの性能は大幅に上昇したけど、コンピュータは**無限**は扱えないので必ずどこかに上限・下限が存在している
- よほどの事がないと超えることはないかも知れないけど、超えるとオーバーフロー(0に戻る)を起こしたりする
    - 昔のゲームはこれでよくバグっていた
    - 上限・下限を超えた場合に何が起こるのかは言語依存なので要注意
        - 無限扱いになったり、マイナスになったり

### 整数の上限・下限を確認する

In [15]:
x1 = typemax(Int64)

9223372036854775807

In [16]:
x2 = typemax(Int32)

2147483647

In [17]:
x3 = typemin(Int64)

-9223372036854775808

In [18]:
x4 = typemin(Int32)

-2147483648

### 最大値を超えた場合、どうなるのか?
- Juliaの場合

In [19]:
x5 = x1 + 1

-9223372036854775808

### 浮動小数点の最大値・最小値

In [20]:
y1 = typemax(Float64)

Inf

In [21]:
y2 = typemax(Float32)

Inf32

In [22]:
y3 = typemin(Float64)

-Inf

In [23]:
y4 = typemin(Float32)

-Inf32

In [24]:
y5 = y1 + 1.0

Inf

---

## 丸め誤差(Rounding Error)
- 数学的な概念とコンピュータの表現は必ずしも一対一で対応していない
- コンピュータ上、例えば1/3を正確には表現出来ない
    - 本来は.3333...が無限に続くけどコンピュータ上はどこかで丸めている
    - **丸め誤差 (rounding error)** と呼ぶ

In [25]:
a = 1/3

0.3333333333333333

- Juliaではうまいことやっていて3をかけると1に戻るようになっているみたい
    - 細かい仕組みは技術書を確認しましょう

In [26]:
b = a*3
b == 1.0 # bが1.0と等しいかをチェック

true

In [27]:
c = 1/7 # 同じく循環小数

0.14285714285714285

* 有理数を扱いたい場合、Juliaは**Rational**という特別な型を用意している
    * 経済学ではそんなに使いみちがなさそうだけど...

In [28]:
d = 1//7

1//7

In [29]:
typeof(d)

Rational{Int64}

---

## 桁落ち(Truncation Error)
- 桁落ちとはなにか？
    - とても大きい数字(例：1234567890123456789)にとても小さい数字(例：0.000000000000000001)を足すと
    - 1234567890123456789.000000000000000001になるはず
    - しかし、数字の情報量が多すぎてコンピュータ上で正確に表現をすることが出来ず、一部を端折ってしまう
    - **大きい数字と小さい数字を混ぜた計算**には注意が必要
        - 重要：足し合わせる場合、小さい値から積み上げていくと誤差を減らすことが出来る
        - ⬆の方法ですべて解決できるわけではない
- クアルテローニ他(2014)、p.8より例示：
$$
    \frac{(1+x)-1}{x}
$$
を計算してみる
* 当然、答えは1のはずだけど...⬇
    * 10%もズレが生じている
* **教訓**：2つの変数の桁が大幅に異なる場合、注意が必要

In [30]:
a = 1234567890123456789

1234567890123456789

In [31]:
b = 0.000000000000000001

1.0e-18

In [32]:
c = a + b

1.2345678901234568e18

In [33]:
x = 0.000000000000001
((1+x)-1)/x

1.1102230246251565

---

## 参考文献
* A.クアルテローニ/F.サレリ/P.ジェルヴァシオ(2014)『MATLABとOctaveによる科学技術計算』丸善出版
* 石井一夫(2021)『基礎から学ぶJulia』SCC